In [ ]:
# importing statements
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Read the dataset
df_1h = pd.read_csv('btc_4h.csv')
df_1h['datetime'] = pd.to_datetime(df_1h['datetime'])
df_1h.set_index('datetime', inplace=True)

In [ ]:
def generate_moving_averages(dataframe, short_window=50, long_window=200):
    df = dataframe.copy()
    df['Short_MA'] = df['close'].rolling(window=short_window, min_periods=1).mean()
    df['Long_MA'] = df['close'].rolling(window=long_window, min_periods=1).mean()
    return df

In [ ]:
def calculate_additional_metrics(dataframe):
    df = dataframe.copy()

    for col in ['MA_Crossover_Signal']:
        df[col + '_Daily_Return'] = df[col].shift(1) * df['close'].pct_change()

    metrics = {}

    for col in ['MA_Crossover_Signal']:
        cumulative_returns = (1 + df[col + '_Daily_Return']).cumprod() - 1
        metrics[col] = {
            'Cumulative_Returns': cumulative_returns[-1],
            'Gross_Profit': df[df[col + '_Daily_Return'] > 0][col + '_Daily_Return'].sum(),
            'Gross_Loss': df[df[col + '_Daily_Return'] < 0][col + '_Daily_Return'].sum(),
            'Net_Profit': cumulative_returns[-1],
            'Total_Closed_Trades': df[col].diff().abs().sum() / 2,
            'Win_Rate': (df[df[col + '_Daily_Return'] > 0][col + '_Daily_Return'].count() / df[col + '_Daily_Return'].count()) * 100,
            'Max_Drawdown': ((cumulative_returns + 1) / (cumulative_returns + 1).cummax() - 1).min(),
            'Average_Winning_Trade': df[df[col + '_Daily_Return'] > 0][col + '_Daily_Return'].mean(),
            'Average_Losing_Trade': df[df[col + '_Daily_Return'] < 0][col + '_Daily_Return'].mean(),
            'Buy_and_Hold_Return_of_BTC': (df['close'].iloc[-1] / df['close'].iloc[0]) - 1,
            'Largest_Losing_Trade': df[df[col + '_Daily_Return'] == df[df[col + '_Daily_Return'] < 0][col + '_Daily_Return'].min()]['close'].iloc[0],
            'Largest_Winning_Trade': df[df[col + '_Daily_Return'] == df[df[col + '_Daily_Return'] > 0][col + '_Daily_Return'].max()]['close'].iloc[0],
            'Sharpe_Ratio': (df[col + '_Daily_Return'].mean() / df[col + '_Daily_Return'].std()) * (252 ** 0.5),
            'Sortino_Ratio': (df[col + '_Daily_Return'].mean() / df[df[col + '_Daily_Return'] < 0][col + '_Daily_Return'].std()) * (252 ** 0.5),
        }

    return metrics

In [ ]:
# Generate Moving Averages
df_1h = generate_moving_averages(df_1h)

In [ ]:

# Generate Signals based on MA Crossover
df_1h['MA_Crossover_Signal'] = 0
df_1h.loc[df_1h['Short_MA'] > df_1h['Long_MA'], 'MA_Crossover_Signal'] = 1
df_1h.loc[df_1h['Short_MA'] < df_1h['Long_MA'], 'MA_Crossover_Signal'] = -1

In [ ]:
# Calculate additional metrics
additional_metrics = calculate_additional_metrics(df_1h)


In [ ]:
# Print the metrics
for strategy, metrics in additional_metrics.items():
    print(f'Performance Metrics')
    for metric_name, metric_value in metrics.items():
        print(f'{metric_name}: {metric_value:.4f}')

Performance Metrics
Cumulative_Returns: 3.2502
Gross_Profit: 46.8086
Gross_Loss: -44.1556
Net_Profit: 3.2502
Total_Closed_Trades: 49.5000
Win_Rate: 50.7727
Max_Drawdown: -0.7828
Average_Winning_Trade: 0.0103
Average_Losing_Trade: -0.0102
Buy_and_Hold_Return_of_BTC: 1.7610
Largest_Losing_Trade: 37025.3200
Largest_Winning_Trade: 4800.0000
Sharpe_Ratio: 0.2871
Sortino_Ratio: 0.3644


In [ ]:
# Plotting
fig = make_subplots(rows=1, cols=1, subplot_titles=['MA Crossover'])

fig.add_trace(go.Scatter(x=df_1h.index, y=df_1h['close'], mode='lines', name='Closing Price', line=dict(color='blue', width=1)))

signal_colors = {'MA_Crossover_Signal': 'green'}
signal_titles = {'MA_Crossover_Signal': 'MA Crossover Signals'}

for col in ['MA_Crossover_Signal']:
    buy_signals = df_1h.index[df_1h[col] == 1]
    sell_signals = df_1h.index[df_1h[col] == -1]

    if not buy_signals.empty:
        fig.add_trace(go.Scatter(x=buy_signals, y=df_1h.loc[buy_signals, 'close'],
                                 mode='markers', name=f'{col} Buy Signal', marker=dict(symbol='triangle-up', size=5, color="green")))

    if not sell_signals.empty:
        fig.add_trace(go.Scatter(x=sell_signals, y=df_1h.loc[sell_signals, 'close'],
                                 mode='markers', name=f'{col} Sell Signal', marker=dict(symbol='triangle-down', size=5, color="red")))

fig.update_layout(
    title='MA Crossover Signals on 4-hour Dataset',
    showlegend=False,
)

fig.show()
